# Домашняя работа
Взять boston house-prices datase (sklearn.datasets.load_boston). Возмите 7 любых  регрессоров (попробовать разные алгоритмы, поподбирать параметры, вывести итоговое качество).

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from scipy.stats import uniform #

In [2]:
boston = load_boston()
#Описание атрибутов датасета
DESCR = {'CRIM':'Криминальный уровень преступности на душу населения',
         'ZN':'Доля жилой земли, зонированной для участков площадью более 25 000 кв. футов.',
         'INDUS':'Доля акров неторгового бизнеса',
         'CHAS':'Фиктивная переменная (= 1, если тракт ограничивает реку; 0 в противном случае)',
         'NOX':'Концентрация оксидов азота NOX (частей на 10 млн.)',
         'RM':'Cреднее количество комнат в жилом помещении',
         'AGE':'Доля занятых квартир в домах построенных до 1940 года',
         'DIS':'Взвешенные расстояния до пяти Бостонских центров занятости',
         'RAD':'Индекс доступности радиальных магистралей',
         'TAX':'Cтавка налога на недвижимость с полной стоимостью за 10 000 долларов США',
         'PTRATIO':'Cоотношение учащихся и учителей',
         'B':'1000(Bk - 0.63)^2, где Bk-доля чернокожих',
         'LSTAT':'% населения с низким статусом',
         'MEDV':'Медианная стоимость домов, занятых владельцами, в $1000'
         }

In [3]:
#Общая информация по датасету
data_boston = pd.concat([pd.DataFrame(boston.data, columns = boston.feature_names), 
                         pd.DataFrame({'MEDV':boston.target})], axis = 1)
data_boston.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


In [4]:
# 7 Регрессоров для опробации
models = []
#1
from sklearn.tree import DecisionTreeRegressor #Дерево решений
models.append({'name'  : "DTR", 
               "model" : DecisionTreeRegressor(), 
               "params": {"criterion": ["mse", "friedman_mse", "mae", "poisson"],
                          "splitter" : ["best", "random"],
                          "max_depth": [1,3,5,7,9,11]}})
#2
from sklearn.ensemble import RandomForestRegressor #Случайный лес
models.append({'name'  : "RFR", 
               "model" : RandomForestRegressor(), 
               "params": {"n_estimators": [10,25,50,100,150,200],
                          "criterion"   : ["mse", "mae"],
                          "max_depth"   : [1,3,5,7,9,11]}})
#3
from sklearn.neighbors import KNeighborsRegressor #К-ближайших соседей
models.append({"name"  : "KNR",
               "model" : KNeighborsRegressor(),
               "params": {"n_neighbors": list(range(1,30)),
                          "weights"    : ["uniform", "distance"],
                          "p"          : [1,2,3]}})
#4
from sklearn.svm import LinearSVR #Линейный метод опртных векторов
models.append({"name"  : "LSVR",
               "model" : LinearSVR(),
               "params": {'C':[0.1, 0.2, 0.3, 0.5, 0.7, 1],
                          "loss": ["epsilon_insensitive", "squared_epsilon_insensitive"],
                          "dual": [True, False]}})
#5
from sklearn.linear_model import SGDRegressor #
models.append({"name"  : "SGDR",
               "model" : SGDRegressor(),
               "params": {"loss"   : ["squared_loss", "huber", "epsilon_insensitive", "squared_epsilon_insensitive"],
                          "penalty": ["l2", "l1", "elasticnet"],
                          "alpha"  : uniform(loc=0, scale=3)}})
#6
from sklearn.linear_model import Ridge #
models.append({"name" : "Ridge",
              "model" : Ridge(), 
              "params": {"alpha" : uniform(loc=0, scale=3), 
                         "solver": ["svd", "cholesky", "lsqr", "sparse_cg", "sag", "saga"]}})
#7
from sklearn.neural_network import MLPRegressor #Нейронная сеть
models.append({"name"  : "MLPR",
               "model" : MLPRegressor(),
               "params": {"activation": ["identity", "logistic", "tanh", "relu"],
                          "solver"    : ["lbfgs", "sgd", "adam"],
                          "alpha" : uniform(loc=0, scale=3)}})

In [6]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV #Используется для поббора оптимальных параметров модели

X_train, X_test, y_train, y_test = train_test_split( data_boston.iloc[:,:-1], 
                                                     data_boston.iloc[:,-1], 
                                                     test_size=0.3, 
                                                     random_state = 42 )

result = []
for item in  models:
    result.append(( item["name"], RandomizedSearchCV(item["model"], item["params"]).fit(X_train, y_train) ))

In [7]:
#Итоговое качество
for item in result:
    print(item[0], item[1].best_score_, item[1].best_params_)

DTR 0.7250205052330319 {'splitter': 'random', 'max_depth': 9, 'criterion': 'mae'}
RFR 0.8096000749857378 {'n_estimators': 100, 'max_depth': 9, 'criterion': 'mse'}
KNR 0.5029705248176051 {'weights': 'uniform', 'p': 1, 'n_neighbors': 6}
LSVR 0.6743331619735907 {'loss': 'squared_epsilon_insensitive', 'dual': False, 'C': 0.2}
SGDR -2.322185602566152 {'alpha': 1.333412809036234, 'loss': 'huber', 'penalty': 'l1'}
Ridge 0.6879694604793952 {'alpha': 0.022780817188248426, 'solver': 'svd'}
MLPR 0.6125438211072944 {'activation': 'relu', 'alpha': 1.864734248053018, 'solver': 'lbfgs'}
